In [ ]:
import spacy
import numpy as np
import re
import itertools
from collections import Counter
import json
import pandas as pd
import torch
import dill as pickle
import time
from data_module.data_preprocessor import preprocess_question, get_label
from collections import Counter
from pandas_ml import ConfusionMatrix

In [ ]:
df_test = pd.read_csv("ir_test_dataset.csv")

In [ ]:
model = torch.load('LSTM_related/lstm_model.model')

In [ ]:
text_field = pickle.load(open("LSTM_related/text_vocab.pkl", "rb"))
label_field = pickle.load(open("LSTM_related/label_vocab.pkl", "rb"))

In [ ]:
def model_prediction(model, text_field, label_field, test_data):
    res = []
    time_predictions = []
    for text in test_data:
        start_time = time.time()
        text = preprocess_question(text, text_field, use_gpu=True)
        model.eval()
        y = model(text)
        label_string = get_label(y, label_field)
        time_predictions.append(time.time() - start_time)
        res.append(label_string)
        del text
        torch.cuda.empty_cache()
    avg_time = np.average(time_predictions)
    # return prediction results and avg time to predict a records
    return res, avg_time

In [ ]:
res_tmp, avg_time_tmp = model_prediction(model, text_field, label_field, df_test.text.values)

In [ ]:
res_tmp # evaluation dataset prediction

In [ ]:
avg_time_tmp

In [ ]:
df_train = pd.read_csv("ir_train_dataset.csv")

In [ ]:
res_tmp_2, avg_time_tmp_2 = model_prediction(model, text_field, label_field, df_train.text.values)

In [ ]:
avg_time_tmp_2

In [ ]:
avg_time_all = (avg_time_tmp + avg_time_tmp_2)/2

In [ ]:
avg_time_all #average time LSTM predict a record

In [ ]:
1/avg_time_all #records per seconds